## Train Relevance Model

Now that the dataset has been [extracted](pdf_text_extraction.ipynb) and [curated](pdf_text_curation.ipynb), we will train the relevance classifier model in this notebook. The model trained is comprised of a transformer model (e.g., BERT) that can be loaded pre-trained on the NQ dataset into the pipeline and then be fine-tuned on the curated data for our specific relevance detection task.

Our pipeline includes components that are provided by the FARM library. FARM is a framework which facilitates transfer learning tasks for BERT based models. Documentation for FARM is available here: https://farm.deepset.ai.

In [1]:
import os
import config
import zipfile
import pathlib
from io import BytesIO

import pandas as pd

from dotenv import load_dotenv

from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score

from src.models import FARMTrainer
from src.data.s3_communication import S3Communication, S3FileType

from config_farm_train import (
    FileConfig,
    ModelConfig,
    MLFlowConfig,
    TrainingConfig,
    TokenizerConfig,
    ProcessorConfig,
)
from farm.infer import Inferencer

07/15/2022 01:55:32 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
# Load credentials
dotenv_dir = os.environ.get(
    "CREDENTIAL_DOTENV_DIR", os.environ.get("PWD", "/opt/app-root/src")
)
dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

In [3]:
# init s3 connector
s3c = S3Communication(
    s3_endpoint_url=os.getenv("S3_ENDPOINT"),
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    s3_bucket=os.getenv("S3_BUCKET"),
)

#### Set parameters

Before starting training, parameters for each component of the training pipeline must be set. For this we create `config` objects which hold these parameters. Default values have already been set but they can be easily changed. To do so, you can manually update the parameters in the corresponding config file:

`aicoe-osc-demo/notebooks/demo2/config_farm_train.py`

In [4]:
# Settings data files and checkpoints parameters
file_config = FileConfig(config.EXPERIMENT_NAME)

# Settings for the processor component
processor_config = ProcessorConfig(config.EXPERIMENT_NAME)

# Settings for the tokenizer
tokenizer_config = TokenizerConfig(config.EXPERIMENT_NAME)
# NOTE: specifically for tokenizer, we need to ensure root dir is a string
tokenizer_config.root = str(tokenizer_config.root)

# Settings for the model
model_config = ModelConfig(config.EXPERIMENT_NAME)

# Settings for training
train_config = TrainingConfig(config.EXPERIMENT_NAME)

# Settings for training
mlflow_config = MLFlowConfig(config.EXPERIMENT_NAME)

We can check the value for some parameters:

In [5]:
print(f"Experiment_name: \n {file_config.experiment_name} \n")
print(f"Data directory: \n {file_config.data_dir} \n")
print(f"Curated dataset path: \n {file_config.curated_data} \n")
print(f"Split train/validation ratio: \n{file_config.dev_split} \n")
print(f"Training dataset path: \n {file_config.train_filename} \n")
print(f"Validation dataset path: \n {file_config.dev_filename} \n")
print(f"Directory where trained model is saved: \n {file_config.saved_models_dir} \n")

Experiment_name: 
 test_cdp2 

Data directory: 
 /opt/app-root/src/aicoe-osc-demo-MichaelTiemannOSC/data 

Curated dataset path: 
 /opt/app-root/src/aicoe-osc-demo-MichaelTiemannOSC/data/curation/esg_TEXT_dataset.csv 

Split train/validation ratio: 
0.2 

Training dataset path: 
 /opt/app-root/src/aicoe-osc-demo-MichaelTiemannOSC/data/kpi_train_split.csv 

Validation dataset path: 
 /opt/app-root/src/aicoe-osc-demo-MichaelTiemannOSC/data/kpi_val_split.csv 

Directory where trained model is saved: 
 /opt/app-root/src/aicoe-osc-demo-MichaelTiemannOSC/models/RELEVANCE 



In [6]:
print(f"Max number of tokens per example: {processor_config.max_seq_len} \n")

Max number of tokens per example: 128 



In [7]:
print(f"Use GPU: {train_config.use_cuda} \n")

Use GPU: True 



In [8]:
print(f"Learning_rate: {train_config.learning_rate} \n")
print(f"Number of epochs for fine tuning: {train_config.n_epochs} \n")
print(f"Batch size: {train_config.batch_size} \n")
print(f"Perform Cross validation: {train_config.run_cv} \n")

Learning_rate: 1e-05 

Number of epochs for fine tuning: 1 

Batch size: 1 

Perform Cross validation: False 



## Load Pretrained Model

We already have a trained relevance classifier on Google's large NQ dataset. We download it and then save it in the following directory: `file_config.saved_models_dir / "relevance_roberta"`

In [9]:
%env AUTOMATION = 1

env: AUTOMATION=1


In [10]:
# When running in Automation using Elyra and Kubeflow Pipelines,
# set AUTOMATION = 1 as an environment variable
if os.getenv("AUTOMATION"):
    # extracted pdfs
    if not os.path.exists(config.BASE_EXTRACTION_FOLDER):
        config.BASE_EXTRACTION_FOLDER.mkdir(parents=True, exist_ok=True)

    # curated pdfs
    if not os.path.exists(config.BASE_CURATION_FOLDER):
        config.BASE_CURATION_FOLDER.mkdir(parents=True, exist_ok=True)

    # processed data
    if not os.path.exists(config.BASE_PROCESSED_DATA):
        config.BASE_PROCESSED_DATA.mkdir(parents=True, exist_ok=True)

    # load dir
    if not os.path.exists(model_config.load_dir):
        pathlib.Path(model_config.load_dir).mkdir(parents=True, exist_ok=True)

    # download extracted pdfs from s3
    s3c.download_files_in_prefix_to_dir(
        config.BASE_EXTRACTION_S3_PREFIX,
        config.BASE_EXTRACTION_FOLDER,
    )
    # download curated pdfs from s3
    s3c.download_files_in_prefix_to_dir(
        config.BASE_CURATION_S3_PREFIX,
        config.BASE_CURATION_FOLDER,
    )

    # download the pretrained model if needed
    model_root = pathlib.Path(model_config.load_dir).parent
    model_rel_zip = pathlib.Path(model_root, "relevance_roberta.zip")

    if not os.path.exists(model_rel_zip):
        s3c.download_file_from_s3(
            model_rel_zip, config.CHECKPOINT_S3_PREFIX, "relevance_roberta.zip"
        )

    with zipfile.ZipFile(pathlib.Path(model_root, "relevance_roberta.zip"), "r") as z:
        z.extractall(model_root)

In [11]:
file_config.data_type = "Text"
print(f"Data type: \n {file_config.data_type} \n")

Data type: 
 Text 



We need to load this model in our pipeline to fine-tune a relevance classifier on our specific ESG curated dataset. For this we have to set the parameter `model_config.load_dir` to be the directory where we saved our first checkpoint. We can check that this is set:

In [12]:
print(f"NQ checkpoint directory: {model_config.load_dir}")

NQ checkpoint directory: /opt/app-root/src/aicoe-osc-demo-MichaelTiemannOSC/models/relevance_roberta


## Fine-tune on curated ESG data

Once all the parameters are set, a `FARMTrainer` object can be instantiated by passing all the configuration objects

In [13]:
# init farm trainer
farm_trainer = FARMTrainer(
    file_config=file_config,
    tokenizer_config=tokenizer_config,
    processor_config=processor_config,
    model_config=model_config,
    training_config=train_config,
    mlflow_config=mlflow_config,
)

Call the method `run()` to start training

In [14]:
farm_trainer.run()

07/15/2022 01:55:38 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: True
07/15/2022 01:55:38 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'RobertaTokenizer'
07/15/2022 01:55:38 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
07/15/2022 01:55:38 - INFO - farm.data_handler.data_silo -   Loading train set from: /opt/app-root/src/aicoe-osc-demo-MichaelTiemannOSC/data/kpi_train_split.csv 
07/15/2022 01:55:38 - INFO - farm.data_handler.data_silo -   Got ya 7 parallel workers to convert 156 dictionaries to pytorch datasets (chunksize = 6)...
07/15/2022 01:55:38 - INFO - farm.data_handler.data_silo -    0    0    0    0    0    0    0 
07/15/2022 01:55:38 - INFO - farm.data_handler.data_silo -   /|\  /w\  /|\  /|\  /|\  /|\  /w\
07/15/2022

1.0

At the end of the training process, the model and the processor vocabulary are saved into the directory `file_config.saved_models_dir`

In [15]:
file_config.saved_models_dir

'/opt/app-root/src/aicoe-osc-demo-MichaelTiemannOSC/models/RELEVANCE'

In [16]:
!ls -al $file_config.saved_models_dir

total 488352
drwxrwsr-x. 2 1000630000 1000630000      4096 Jul  9 08:41 .
drwxrwsr-x. 8 1000630000 1000630000      4096 Jul 15 01:52 ..
-rw-rw-r--. 1 1000630000 1000630000 498669047 Jul 15 01:56 language_model.bin
-rw-rw-r--. 1 1000630000 1000630000       562 Jul 15 01:56 language_model_config.json
-rw-rw-r--. 1 1000630000 1000630000    456318 Jul 15 01:56 merges.txt
-rw-rw-r--. 1 1000630000 1000630000      7489 Jul 15 01:56 prediction_head_0.bin
-rw-rw-r--. 1 1000630000 1000630000       321 Jul 15 01:56 prediction_head_0_config.json
-rw-rw-r--. 1 1000630000 1000630000       789 Jul 15 01:56 processor_config.json
-rw-rw-r--. 1 1000630000 1000630000       772 Jul 15 01:56 special_tokens_map.json
-rw-rw-r--. 1 1000630000 1000630000       240 Jul 15 01:56 tokenizer_config.json
-rw-rw-r--. 1 1000630000 1000630000    898822 Jul 15 01:56 vocab.json


## Cross-validation

To better estimate the performance of the model on new data, it is recommended to perform k-folds cross validation (CV). CV works as follows:

- Split the entire data randomly into k folds (usually 5 to 10)
- Fit the model using the K — 1 folds and validate the model using the remaining Kth fold and save the scores
- Repeat until every K-fold serve as the test set and average the saved scores

`FARMTrainer` includes this features. To perform 3-fold CV proceed as follows:

In [17]:
train_config.run_cv = True
train_config.xval_folds = 2
train_config.n_epochs = 1
train_config.batch_size = 8

In [18]:
farm_trainer = FARMTrainer(
    file_config=file_config,
    tokenizer_config=tokenizer_config,
    model_config=model_config,
    processor_config=processor_config,
    training_config=train_config,
    mlflow_config=mlflow_config,
)

In [19]:
farm_trainer.run()

07/15/2022 01:56:16 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: True
07/15/2022 01:56:16 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'RobertaTokenizer'
07/15/2022 01:56:17 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
07/15/2022 01:56:17 - INFO - farm.data_handler.data_silo -   Loading train set from: /opt/app-root/src/aicoe-osc-demo-MichaelTiemannOSC/data/kpi_train_split.csv 
07/15/2022 01:56:17 - INFO - farm.data_handler.data_silo -   Got ya 7 parallel workers to convert 156 dictionaries to pytorch datasets (chunksize = 6)...
07/15/2022 01:56:17 - INFO - farm.data_handler.data_silo -    0    0    0    0    0    0    0 
07/15/2022 01:56:17 - INFO - farm.data_handler.data_silo -   /|\  /w\  /|\  /|\  /|\  /|\  /w\
07/15/2022

**NOTE:** CV mode does not save a checkpoint, it is only used for validation

## Model Performance Metrics

In this section, we will quantify the performance of the fine tuned model on our dataset. Specifically, we will calculate the precision, recall, and f1-score. 

In [20]:
# load test set
test_data = pd.read_csv(file_config.dev_filename, index_col=0)
test_data.head()

,label,text,text_b
154,1,What is the target metric in the target year?,Target year 2029 Targeted reduction from base ...
167,1,What is the target reduction amount?,Target year 2025 Targeted reduction from base ...
32,1,Provide your base year and base year scope2 (l...,Scope 2 (location-based) Base year start Janua...
115,1,What is the base year start date for scope 2 (...,Scope 2 (market-based) Base year start January...
59,1,What is the End Date of the CDP report published?,Bayer AG - Climate Change 2021


In [21]:
# get predictions from current model
model = Inferencer.load(file_config.saved_models_dir)

result = model.inference_from_file(file_config.dev_filename)
results = [d for r in result for d in r["predictions"]]
preds = [int(r["label"]) for r in results]

test_data["pred"] = preds

07/15/2022 01:56:43 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
07/15/2022 01:56:46 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
07/15/2022 01:56:46 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
07/15/2022 01:56:46 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
07/15/2022 01:56:46 - INFO - farm.modeling.prediction_head -   Loading prediction head from /opt/app-root/src/aicoe-osc-demo-MichaelTiemannOSC/models/RELEVANCE/prediction_head_0.bin
07/15/2022 01:56:46 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'RobertaTokenizer'
07/15/2022 01:56:46 - INFO - farm.data_handler.processor -   Initialized processor without tasks. Supply `metric` and `label_list` to the constructor for using the default task or add a custom task later via processor.add_task()
07/15/2022 01:56

In [22]:
# evalute performance
groups = test_data.groupby("text")
scores = {}
for group, data in groups:
    pred = data.pred
    true = data.label
    scores[group] = {}
    scores[group]["accuracy"] = accuracy_score(true, pred)
    scores[group]["f1_score"] = f1_score(true, pred)
    scores[group]["recall_score"] = recall_score(true, pred)
    scores[group]["precision_score"] = precision_score(true, pred)
    scores[group]["support"] = len(pred)

/opt/app-root/lib64/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/app-root/lib64/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/app-root/lib64/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/app-root/lib64/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetri

In [23]:
# kpi wise performance metrics
scores_df = pd.DataFrame(scores)
scores_df.head()

,Did you have an emissions target that was active in the reporting year?,Provide your base year and base year scope 1 emissions.,Provide your base year and base year scope2 (location-based) emissions.,Provide your base year and base year scope2 (market-based) emissions.,What is the % change in absolute Scope 3 emissions?,What is the End Date of the CDP report published?,What is the Start Date of the CDP report published?,What is the absolute target name?,What is the base year end date for scope 1 emissions?,What is the base year start date for scope 2 (location-based) emissions?,What is the base year start date for scope 2 (market-based) emissions?,What is the company name?,What is the intensity figure in reporting year?,What is the target base year value?,What is the target metric in the target year?,What is the target metric?,What is the target reduction amount?,What is the target scope?,What is the target year?,What is the year the target was set?
accuracy,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f1_score,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
recall_score,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
precision_score,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
support,1.0,3.0,4.0,4.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,4.0,1.0,1.0,2.0


In [24]:
# save results locally
if not os.getenv("AUTOMATION"):
    scores_df.to_csv(file_config.model_performance_metrics_file)

## Save model to s3

Great, we have a fine tuned model at this point. We will now save this model as well as its performance metrics to s3.

In [25]:
# upload performance files to s3
s3c.upload_df_to_s3(
    scores_df,
    s3_prefix=config.BASE_SAVED_MODELS_S3_PREFIX,
    s3_key="relevance_scores.csv",
    filetype=S3FileType.CSV,
)

{'ResponseMetadata': {'RequestId': 'EXHXTGM92TWNCAN6',
  'HostId': '+FvhWwfEf4g3045iSQ6d0XSPtBk25bdI56w9xDIvP1jujBz8juySC1A2JT6zb0APcEUfEomxpdI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '+FvhWwfEf4g3045iSQ6d0XSPtBk25bdI56w9xDIvP1jujBz8juySC1A2JT6zb0APcEUfEomxpdI=',
   'x-amz-request-id': 'EXHXTGM92TWNCAN6',
   'date': 'Fri, 15 Jul 2022 01:57:28 GMT',
   'etag': '"772812863cfff927dea06a877a4399f4"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"772812863cfff927dea06a877a4399f4"'}

In [26]:
buffer = BytesIO()
with zipfile.ZipFile(buffer, 'a') as z:
    for dirname, _, files in os.walk(file_config.saved_models_dir):
        for f in files:
            f_path = os.path.join(dirname, f)
            with open (f_path, 'rb') as file_content:
                z.writestr(f"RELEVANCE/{f}", file_content.read())

In [27]:
buffer.seek(0)
# upload model to s3
s3c._upload_bytes(
    buffer_bytes=buffer,
    prefix=config.BASE_SAVED_MODELS_S3_PREFIX,
    key="RELEVANCE.zip"
)

{'ResponseMetadata': {'RequestId': 'PC58HQPZR7B0Z5ZW',
  'HostId': 'sDtv4RsMG25OZ8h7vTW8fIyxs6ghVQfvjS91UY/1jfmr8P+v6eMfikm9T1rkgKHjEoAaYxG/0Nk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'sDtv4RsMG25OZ8h7vTW8fIyxs6ghVQfvjS91UY/1jfmr8P+v6eMfikm9T1rkgKHjEoAaYxG/0Nk=',
   'x-amz-request-id': 'PC58HQPZR7B0Z5ZW',
   'date': 'Fri, 15 Jul 2022 01:57:30 GMT',
   'etag': '"0447a2f82b2bd63f003a9ca15e1b59c6"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"0447a2f82b2bd63f003a9ca15e1b59c6"'}

# Conclusion

In this notebook, we developed a model that can be used for finding relevant paragraphs for a KPI question, given a list of paragraphs from climate report PDFs. With this model in place, we can go ahead to training the kpi extraction model.